# Implement Minimax and Monte Carlo Tree Search (Total 40 points)

In [ ]:
from collections import namedtuple, Counter, defaultdict
import random
import math
import functools
import numpy as np

## Define Game

We start with defining the abstract class `Game`, for turn-taking *n*-player games.

We rely on, but do not define yet, the concept of a `state` of the game; we'll see later how individual games define states. For now, all we require is that a state has a `state.to_move` attribute, which gives the name of the player whose turn it is. ("Name" will be something like `'X'` or `'O'` for tic-tac-toe.)

We also define `play_game`, which takes a game and a dictionary of  `{player_name: strategy_function}` pairs, and plays out the game, on each turn checking `state.to_move` to see whose turn it is, and then getting the strategy function for that player and applying it to the game and the state to get a move.

In [ ]:
class Game:
    """A game is similar to a problem, but it has a terminal test instead of a goal test, and a utility for each terminal state.
    To create a game, subclass this class and implement `actions`, `result`, `is_terminal`, and `utility`.
    You will also need to set the .initial attribute to the initial state; this can be done in the constructor.
    """
    def actions(self, state):
        """Return a collection of the allowable moves from this state."""
        raise NotImplementedError

    def result(self, state, move):
        """Return the state that results from making a move from a state."""
        raise NotImplementedError

    def is_terminal(self, state):
        """Return True if this is a final state for the game."""
        return not self.actions(state)

    def utility(self, state, player):
        """Return the value of this final state to player."""
        raise NotImplementedError

    def to_move(self, state):
        """Return the player whose move it is in this state."""
        return state.to_move

def play_game(game, strategies: dict, verbose=False):
    """Play a turn-taking game.
    `strategies` is a {player_name: function} dict, where function(state, game) is used to get the player's move.
    """
    state = game.initial
    while not game.is_terminal(state):
        player = state.to_move
        move = strategies[player](game, state)
        state = game.result(state, move)
        if verbose:
            print('Player', player, 'move:', move)
            print(state)
    return state

## A Simple Game: Tic-Tac-Toe

We have the notion of an abstract game. Now it is time to define a simple real game: tic-tac-toe. Moves are `(x, y)` pairs denoting squares, where `(0, 0)` is the top left, and `(2, 2)` is the bottom right (on a board of size `height=width=3`).

States in tic-tac-toe (and other games) will be represented as a `Board`, which is a subclass of `defaultdict` that in general will consist of `{(x, y): contents}` pairs, for example `{(0, 0): 'X', (1, 1): 'O'}` might be the state of the board after two moves. Besides the contents of squares, a board also has some attributes:
- `.to_move` to name the player whose move it is;
- `.width` and `.height` to give the size of the board (both 3 in tic-tac-toe, but other numbers in related games);
- possibly other attributes, as specified by keywords.

As a `defaultdict`, the `Board` class has a `__missing__` method, which returns `empty` for squares that have no been assigned but are within the `width` × `height` boundaries, or `off` otherwise. The class has a `__hash__` method, so instances can be stored in hash tables.

In [ ]:
class Board(defaultdict):
    """A board has the player to move, a cached utility value, and a dict of {(x, y): player} entries, where player is 'X' or 'O'."""

    empty = '.'
    off = '#'

    def __init__(self, width=3, height=3, to_move=None, **kwds):
        self.__dict__.update(width=width, height=height, to_move=to_move, **kwds)

    def new(self, changes: dict, **kwds) -> 'Board':
        "Given a dict of {(x, y): contents} changes, return a new Board with the changes."
        board = Board(width=self.width, height=self.height, **kwds)
        board.update(self)
        board.update(changes)
        return board

    def __missing__(self, loc):
        x, y = loc
        if 0 <= x < self.width and 0 <= y < self.height:
            return self.empty
        else:
            return self.off

    def __hash__(self):
        return hash(tuple(sorted(self.items()))) + hash(self.to_move)

    def __repr__(self):
        def row(y):
            return ' '.join(self[x, y] for x in range(self.width))
        return '\n'.join(map(row, range(self.height))) +  '\n'

In [ ]:
class TicTacToe(Game):
    """Play TicTacToe on an `height` by `width` board, needing `k` in a row to win.
    'X' plays first against 'O'."""

    def __init__(self, height=3, width=3, k=3):
        self.k = k # k in a row
        self.squares = {(x, y) for x in range(width) for y in range(height)}
        self.initial = Board(height=height, width=width, to_move='X', utility=0)

    def actions(self, board):
        """Legal moves are any square not yet taken."""
        return self.squares - set(board)

    def result(self, board, square):
        """Place a marker for current player on square."""
        player = board.to_move
        board = board.new({square: player}, to_move=('O' if player == 'X' else 'X'))
        win = k_in_row(board, player, square, self.k)
        board.utility = (0 if not win else +1 if player == 'X' else -1)
        return board

    def utility(self, board, player):
        """Return the value to player; 1 for win, -1 for loss, 0 otherwise."""
        return board.utility if player == 'X' else -board.utility

    def is_terminal(self, board):
        """A board is a terminal state if it is won or there are no empty squares."""
        return board.utility != 0 or len(self.squares) == len(board)

    def display(self, board):
        print(board)

def k_in_row(board, player, square, k):
    """True if player has k pieces in a line through square."""
    def in_row(x, y, dx, dy):
        return 0 if board[x, y] != player else 1 + in_row(x + dx, y + dy, dx, dy)
    return any(in_row(*square, dx, dy) + in_row(*square, -dx, -dy) - 1 >= k
               for (dx, dy) in ((0, 1), (1, 0), (1, 1), (1, -1)))

## Minimax-Based Game Search Algorithm

The following minimax-based game search algorithms take two inputs: `game`the game we are playing, and `state`, the current state of the game.
It returns a `(value, move)` pair, where `value` is the utility that the algorithm computes for the player whose turn it is to move, and `move` is the move itself.

First we define `minimax_search`, which exhaustively searches the game tree to find an optimal move (assuming both players play optimally), and `alphabeta_search`, which does the same computation, but prunes parts of the tree that could not possibly have an affect on the optimnal move.  

## Complete `minimax_search` below (20 points).

In [ ]:
infinity = math.inf

def minimax_search(game, state):
    """Search game tree to determine best move; return (value, move) pair."""

    player = state.to_move

    def max_value(state):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -infinity, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a))
            if v2 > v:
                v, move = v2, a
        return v, move

    def min_value(state):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = +infinity, None
        for a in game.actions(state):
            ####################Write your code here ################################# (20 points)
            # Hint: only three lines are sufficiently; take a close look at the max_value function above; It is very similar
            # Hint: You can also refer to alphabeta_search below, which is a more complex version of minimax_search
            v2, _ = min_value(game.result(state, a))
            if v2 < v:
                v, move = v2, a
            ###########################End code#############################################
        return v, move

    return max_value(state)

def alphabeta_search(game, state):
    """Search game to determine best action; use alpha-beta pruning.
    As in the AI textbook 4th edition [Figure 5.7], this version searches all the way to the leaves."""

    player = state.to_move

    def max_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -infinity, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a), alpha, beta)
            if v2 > v:
                v, move = v2, a
                alpha = max(alpha, v)
            if v >= beta:
                return v, move
        return v, move

    def min_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = +infinity, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a), alpha, beta)
            if v2 < v:
                v, move = v2, a
                beta = min(beta, v)
            if v <= alpha:
                return v, move
        return v, move

    return max_value(state, -infinity, +infinity)

## Players

We need an interface for players. I'll represent a player as a `callable` that will be passed two arguments: `(game, state)` and will return a `move`.
The function `player` creates a player out of a search algorithm, but you can create your own players as functions, as is done with `random_player` below:

In [ ]:
def random_player(game, state):
    return random.choice(list(game.actions(state)))

def player(search_algorithm):
    """A game player who uses the specified search algorithm"""
    return lambda game, state: search_algorithm(game, state)[1]

# Playing a Game

We're ready to play a game. I'll set up a match between a `random_player` (who chooses randomly from the legal moves) and a `player(alphabeta_search)` (who makes the optimal alpha-beta move; practical for tic-tac-toe, but not for large games). The `player(alphabeta_search)` will never lose, but if `random_player` is lucky, it will be a tie.

In [ ]:
play_game(TicTacToe(), dict(X=player(alphabeta_search), O=random_player), verbose=True).utility

Player X move: (0, 1)
. . .
X . .
. . .

Player O move: (0, 0)
O . .
X . .
. . .

Player X move: (1, 1)
O . .
X X .
. . .

Player O move: (1, 2)
O . .
X X .
. O .

Player X move: (2, 1)
O . .
X X X
. O .



1

The alpha-beta player will never lose, but sometimes the random player can stumble into a draw. When two optimal (alpha-beta or minimax) players compete, it will always be a draw:

In [ ]:
play_game(TicTacToe(), dict(X=player(alphabeta_search), O=player(minimax_search)), verbose=True).utility

Player X move: (0, 1)
. . .
X . .
. . .

Player O move: (1, 2)
. . .
X . .
. O .

Player X move: (1, 1)
. . .
X X .
. O .

Player O move: (2, 1)
. . .
X X O
. O .

Player X move: (0, 0)
X . .
X X O
. O .

Player O move: (1, 0)
X O .
X X O
. O .

Player X move: (0, 2)
X O .
X X O
X O .



1

# Heuristic Cutoffs

In [ ]:
def cutoff_depth(d):
    """A cutoff function that searches to depth d."""
    return lambda game, state, depth: depth > d

def cache1(function):
    "Like lru_cache(None), but only considers the first argument of function."
    cache = {}
    def wrapped(x, *args):
        if x not in cache:
            cache[x] = function(x, *args)
        return cache[x]
    return wrapped

def h_alphabeta_search(game, state, cutoff=cutoff_depth(6), h=lambda s, p: 0):
    """Search game to determine best action; use alpha-beta pruning.
    As in [Figure 5.7], this version searches all the way to the leaves."""

    player = state.to_move

    @cache1
    def max_value(state, alpha, beta, depth):
        if game.is_terminal(state):
            return game.utility(state, player), None
        if cutoff(game, state, depth):
            return h(state, player), None
        v, move = -infinity, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a), alpha, beta, depth+1)
            if v2 > v:
                v, move = v2, a
                alpha = max(alpha, v)
            if v >= beta:
                return v, move
        return v, move

    @cache1
    def min_value(state, alpha, beta, depth):
        if game.is_terminal(state):
            return game.utility(state, player), None
        if cutoff(game, state, depth):
            return h(state, player), None
        v, move = +infinity, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a), alpha, beta, depth + 1)
            if v2 < v:
                v, move = v2, a
                beta = min(beta, v)
            if v <= alpha:
                return v, move
        return v, move

    return max_value(state, -infinity, +infinity, 0)

In [ ]:
play_game(TicTacToe(), {'X':player(h_alphabeta_search), 'O':player(h_alphabeta_search)}, verbose=True).utility

Player X move: (0, 1)
. . .
X . .
. . .

Player O move: (2, 1)
. . .
X . O
. . .

Player X move: (1, 2)
. . .
X . O
. X .

Player O move: (0, 0)
O . .
X . O
. X .

Player X move: (1, 1)
O . .
X X O
. X .

Player O move: (1, 0)
O O .
X X O
. X .

Player X move: (2, 0)
O O X
X X O
. X .

Player O move: (0, 2)
O O X
X X O
O X .

Player X move: (2, 2)
O O X
X X O
O X X



0

In [ ]:
class CountCalls:
    """Delegate all attribute gets to the object, and count them in ._counts"""
    def __init__(self, obj):
        self._object = obj
        self._counts = Counter()

    def __getattr__(self, attr):
        "Delegate to the original object, after incrementing a counter."
        self._counts[attr] += 1
        return getattr(self._object, attr)

def report(game, searchers):
    for searcher in searchers:
        game = CountCalls(game)
        searcher(game, game.initial)
        print('Result states: {:7,d}; Terminal tests: {:7,d}; for {}'.format(
            game._counts['result'], game._counts['is_terminal'], searcher.__name__))

report(TicTacToe(), (alphabeta_search, h_alphabeta_search))

Result states:  23,177; Terminal tests:  23,178; for alphabeta_search
Result states:   3,838; Terminal tests:   2,423; for h_alphabeta_search


# Monte Carlo Tree Search

## Complete `monte_carlo_tree_search` below (20 points).

In [ ]:
class MCT_Node:
    """Node in the Monte Carlo search tree, keeps track of the children states."""

    def __init__(self, parent=None, state=None, U=0, N=0):
        self.__dict__.update(parent=parent, state=state, U=U, N=N)
        self.children = {}
        self.actions = None


def ucb(n, C=1.4):
    """Upper confidence bound
    """
    return np.inf if n.N == 0 else n.U / n.N + C * np.sqrt(np.log(n.parent.N) / n.N)

def monte_carlo_tree_search(state, game, N=1000):
    def select(n):
        """select a leaf node in the tree"""
        if n.children:
            return select(max(n.children.keys(), key=ucb))
        else:
            return n

    def expand(n):
        """expand the leaf node by adding all its children states"""
        if not n.children and not game.is_terminal(n.state):
            n.children = {MCT_Node(state=game.result(n.state, action), parent=n): action
                          for action in game.actions(n.state)}
        return select(n)

    def simulate(game, state):
        """simulate the utility of current state by random picking a step"""
        player = game.to_move(state)
        while not game.is_terminal(state):
            action = random.choice(list(game.actions(state)))
            state = game.result(state, action)
        v = game.utility(state, player)
        return -v

    def backprop(n, utility):
        """passing the utility back to all parent nodes"""
        if utility > 0:
            n.U += utility
        # if utility == 0:
        #     n.U += 0.5
        n.N += 1
        if n.parent:
            backprop(n.parent, -utility)

    root = MCT_Node(state=state)

    for _ in range(N):
        leaf = select(root)
        #############################Write your code here#################################### (10 points)
        # Hint: One single line; call the expand function define above and return child for the simulate function below
        child = expand(leaf)
        ###############################End code#########################################################
        result = simulate(game, child.state)
        #############################Write your code here#################################### (10 points)
        # Hint: One single line; call the backprop function define above; you only need to specify the two arguments for the backprop function
        backprop(child, result)
        ###############################End code#########################################################

    max_state = max(root.children, key=ucb)

    return root.children.get(max_state)

def mcts_player(game, state):
    return monte_carlo_tree_search(state, game)

In [ ]:
play_game(TicTacToe(), {'X': mcts_player, 'O': random_player}, verbose=True).utility

Player X move: (2, 0)
. . X
. . .
. . .

Player O move: (1, 1)
. . X
. O .
. . .

Player X move: (1, 0)
. X X
. O .
. . .

Player O move: (2, 1)
. X X
. O O
. . .

Player X move: (0, 0)
X X X
. O O
. . .



1

In [ ]:
play_game(TicTacToe(), {'X': mcts_player, 'O': player(h_alphabeta_search)}, verbose=True).utility

Player X move: (1, 1)
. . .
. X .
. . .

Player O move: (0, 0)
O . .
. X .
. . .

Player X move: (2, 1)
O . .
. X X
. . .

Player O move: (0, 1)
O . .
O X X
. . .

Player X move: (0, 2)
O . .
O X X
X . .

Player O move: (2, 0)
O . O
O X X
X . .

Player X move: (1, 0)
O X O
O X X
X . .

Player O move: (1, 2)
O X O
O X X
X O .

Player X move: (2, 2)
O X O
O X X
X O X



0